# Hypothesis

In a comparison of neural network architectures, a combination of **one or more convolutional layers** in front of a **radial basis function network** will optimally predict the composition of a sample melt glass bead from a LIBS spectrum (as compared with purely convolutional layers, purely "standard" layers, e.g. fitnet). 

## Metrics

**Optimal prediction** consists of the following metrics:
- Accuracy of prediction on testing set
  - SSE for all analytes
  - MSE for all analytes
  - MAPE for all analytes
  - SSE for each analyte
  - MSE for each analyte
  - MAPE for each analyte
  - Something to do with the $\sigma$ of each analyte?
- Speed of prediction
- Speed of training

# Methods

Choices to be made/variables identified:
- How to split test/validate/train data
  - Ratios?
  - Why have 3 groups vice 2?
- What training algorithm to use (backprop? other?)
- Convolution
  - How many layers?
  - Window size?
  - Step size?
  - Final activation function?
- RBFN
  - How many prototype vectors?
  - How to choose prototype vectors?
- FITNET
  - How many layers?
  - Activation function?
  - Neurons per layer?

Gonna need these libraries

In [18]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Load the data

In [19]:
datafile = 'data/data.csv'

In [20]:
data = np.transpose(np.loadtxt(datafile, delimiter=','))
print(data.shape)

(220, 11738)


In [21]:
input_data = data[:,:11724]
output_data = data[:,11725:]
print(input_data.shape)
print(output_data.shape)

(220, 11724)
(220, 13)


Normalize the input and output

In [22]:
x_norm = MinMaxScaler()
y_norm = MinMaxScaler()

In [23]:
inputs = x_norm.fit_transform(input_data)
outputs = y_norm.fit_transform(output_data)

In [24]:
inputs.shape

(220, 11724)

In [25]:
outputs.shape

(220, 13)

Split these into training, test, validation

In [26]:
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.3)
print(x_train.shape)
print(y_train.shape)

(154, 11724)
(154, 13)


TODO: 
- Change the loader to randomly select test and train groups from each of the samples (so we don't introduce error by oversampling from one bead)
- Switch to a train/validate/test arrangement:
      while validation error > epsilon:
          train on training data
          if runs > max_runs:
              break
      report testing error

# Simple single layer

In [27]:
X = tf.placeholder(tf.float32, [None, x_train.shape[1]])

In [28]:
W = tf.Variable(tf.zeros([x_train.shape[1], y_train.shape[1]]))
b = tf.Variable(tf.zeros([y_train.shape[1]]))
y = tf.nn.softmax(tf.matmul(X, W) + b)

In [29]:
y_ = tf.placeholder(tf.float32, [None, y_train.shape[1]])

In [30]:
mse = tf.reduce_mean((y_ - y) ** 2, reduction_indices=[0, 1])

In [31]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(mse)

In [32]:
sess = tf.InteractiveSession()

In [33]:
tf.global_variables_initializer().run()

In [57]:
for _ in range(1000):
    sess.run(train_step, feed_dict={X: x_train, y_: y_train})

In [58]:
sess.run(mse, feed_dict={X: x_test, y_: y_test})

0.051863749

In [36]:
y_test.shape

(66, 13)

In [101]:
print(W)

<tf.Variable 'Variable_2:0' shape=(11724, 13) dtype=float32_ref>


I want to convert this to give me a simple answer for each iteration, that should let me show progress as training continues.

I also want to set this up so I can watch it in tensorboard.